# Create Slice for Fabric Rack
<br>This is the slice creation for the ```TCP Traffic``` tutorial. This is the first step of the assignment, to create the slice on fabric using fablib.

## 1. Set up the Experiment

In this section you will use the Fablib manager to create a new slice that is composed of 2 nodes for the TCP Traffic Assignment.
### 1.1 Reserve Resources
In the 'EDUKY' site, we will reserve a set of 2 nodes arranged in a line, with one node designated as a server and the other as a client. Each node will have the following specifications: 1 CPU core, 2GB of RAM, and 10GB of storage capacity. Each node will have 1 network card (NICs) to communicate with each other. We will call the server "Node_A" and the card "ND_A" and the client we will be "Node B" and its port "ND_B". The bridge connecting the two nodes is an l2network called "Lan". All nodes will be preloaded with an 'Ubuntu' Linux OS. Upon submission, the slice will be named 'TCP_Traffic_intro'.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "TCP_Traffic_intro"
site = "EDUKY"
print(site)


nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

try:
    # Create a slice
    slice = fablib.new_slice(name=slice_name)
    #first Node
    ND1 = slice.add_node(name="Node_A", site=site)
    ND1.set_capacities(cores=cores, ram=ram, disk=disk)
    ND1.set_image(image)
    Net_ND1 = ND1.add_component(model=nicmodel, name="ND_A").get_interfaces()[0]
    

    #Second Node
    ND2 = slice.add_node(name="Node_B", site=site)
    ND2.set_capacities(cores=cores, ram=ram, disk=disk)
    ND2.set_image(image)
    Net_ND2 = ND2.add_component(model=nicmodel, name="ND_B").get_interfaces()[0] 

    #Lan
    LN = slice.add_l2network(name="Lan", interfaces=[Net_ND1, Net_ND2])
    
    # Submit the slice
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

### 1.2 Set up the Experiment Network
The network configuration part of this slice will connect the nodes with specific IPs such that the nodes are able to communicate with each other. The slice will use the network name to find the corresponding devices (NIC cards) used in the slice creation stage so that we can provide the correct connection in the network, in this case, "Lan".

This slice contains a single subnetwork in IPv4: 10.0.0.0/24 (between Node_A and the Node_B, "Lan"). This slice will provide IP 10.0.0.1 to "Node_A" and 10.0.0.2 to "Node_B" following the guidelines set by the network subnet.

After providing the IPs to the corresponding devices, the cell will output the network status and execute "ip a" to check the connections that are active.

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("10.0.0.0/24")
    Network = 'Lan'
    
    A = slice.get_node('Node_A')
    B = slice.get_node('Node_B')
    
    #A's lan
    A_iface = A.get_interface(network_name=Network)  
    A_iface.ip_addr_add(addr='10.0.0.1', subnet=subnet)
    #B's Lan
    B_iface = B.get_interface(network_name=Network)  
    B_iface.ip_addr_add(addr='10.0.0.2', subnet=subnet)
    
    for node in slice.get_nodes():
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
            if (node.get_name() == "ND_A"):
                A_Device_Name = interface.get_device_name()
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")
        
except Exception as e:
    print(f"Fail: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
we only need to install a couple of tools for our slice, the first of these will be "net-tools" and the second "iperf", these tools will help us monitor traffic inside of the nodes. we also have to set a bandwidth limit to see the correct amount of bandwidth given that in Fabric the lan connections are too good for the experiment we are running, we add a 13 mbit average rate of transmission, 1000mb limit per token and a maximum of 10 kb of data that can be sent in a burst. Since our experiment measures round trip data, we only need to modify one node to get our desired result.

Additionally for this slice, we need to install some software that is only available from a GitHub repository. After we download the script from the GitHub repository, we modify some lines to execute according to our environment. The last step is to install the libraries that the scripts we downloaded from github have, this is the r-studio library: "r-base-core","r-cran-ggplot2" & "r-cran-littler" along with "moreutils". We also use system call to avoid saving necessary tcp data and chmod to turn one of the scripts we downloaded from github into an executable.

In [ ]:
# Install Software
try:
    INTERFACE_NAME_HOLDER = "INTERFACE_NAME"

    for node in slice.get_nodes():        
        node.execute("sudo apt update")
        node.execute("sudo apt install net-tools")
        node.execute("sudo apt install iperf")
        
        #upload scripts to make a graphic
        node.execute(f"sudo tc qdisc add dev {A_Device_Name} root tbf rate 13mbit limit 1000mb burst 10kb")
        node.upload_file("./scripts/ss-data-analysis.R","ss-data-analysis.R")
        node.upload_file("./scripts/ss-output.sh","ss-output.sh")
        node.execute(f"sed -i 's/{INTERFACE_NAME_HOLDER}/{A_Device_Name}/' ss-output.sh")
        node.execute("sudo apt-get -y install moreutils r-base-core r-cran-ggplot2 r-cran-littler;sudo sysctl -w net.ipv4.tcp_no_metrics_save=1;chmod +x ss-output.sh ")

except Exception as e:
    print(f"Fail: {e}")

## Continue to The experiment notebook

Once you have completed this notebook you should be able to continue to the TCP Traffic Notebook. You can either open it on the explorer or click [Here](./TCP_Traffic.ipynb) to open the next notebook.